# Laboratorio 6: Clasificadores de la distancia mínima y 1NN

### **Parte I.** 

Programa y valida el Clasificador de la Distancia Mínima, valídalo con 3 datasets y los siguientes métodos de validación. 

    1.Hold-Out 70/30 estratificado
    2.10-Fold Cross-Validation estratificado
    3.Leave-One-Out.

### **Parte II.** 

Programa y valida el Clasificador 1NN, valídalo con 3 datasets y los siguientes métodos de validación. 

    1.Hold-Out 70/30 estratificado
    2.10-Fold Cross-Validation estratificado
    3.Leave-One-Out.


Medidas de desempeño:

    -Accuracy
    -Matriz de Confusión

In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold, LeaveOneOut
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.datasets import load_iris, load_wine, load_breast_cancer
from sklearn.neighbors import KNeighborsClassifier

In [19]:
# Cargar los datasets 
iris_data = load_iris()
wine_data = load_wine()
cancer_data = load_breast_cancer()

# Convertir datasets a DataFrames
datasets = {
    "Iris": (pd.DataFrame(iris_data['data'], columns=iris_data['feature_names']), pd.Series(iris_data['target'])),
    "Wine": (pd.DataFrame(wine_data['data'], columns=wine_data['feature_names']), pd.Series(wine_data['target'])),
    "Breast Cancer": (pd.DataFrame(cancer_data['data'], columns=cancer_data['feature_names']), pd.Series(cancer_data['target']))
}


In [20]:
# Función de evaluación (confusión y precisión) que incluye todas las etiquetas
def evaluate_model(y_true, y_pred, labels):
    return confusion_matrix(y_true, y_pred, labels=labels), accuracy_score(y_true, y_pred)

# Implementación del clasificador de distancia mínima
def min_distance_classifier(X_train, y_train, X_test):
    centroids = X_train.groupby(y_train).mean()
    return [centroids.sub(x).pow(2).sum(axis=1).idxmin() for _, x in X_test.iterrows()]

In [21]:
# Funciones de validación

def holdout_validation(X, y):
    return train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

def k_fold_cross_validation(X, y, K=10):
    skf = StratifiedKFold(n_splits=K, shuffle=True, random_state=42)
    return [(X.iloc[train], X.iloc[test], y.iloc[train], y.iloc[test]) for train, test in skf.split(X, y)]

def leave_one_out_validation(X, y):
    loo = LeaveOneOut()
    return [(X.iloc[train], X.iloc[test], y.iloc[train], y.iloc[test]) for train, test in loo.split(X)]

# Parte I: Clasificador de Distancia Mínima
for name, (X, y) in datasets.items():
    labels = np.unique(y)  # Definir etiquetas únicas para cada conjunto de datos
    print(f"\n--- Parte I: Clasificador de Distancia Mínima - {name} Dataset ---")
    
    # Validación Hold-Out
    X_train, X_test, y_train, y_test = holdout_validation(X, y)
    y_pred = min_distance_classifier(X_train, y_train, X_test)
    conf_matrix, accuracy = evaluate_model(y_test, y_pred, labels)
    
    print("\nDistancia Mínima - Hold-Out")
    print("Matriz de Confusión:\n", conf_matrix)
    print("Precisión:", accuracy)
    
    # Validación K-Fold
    conf_matrix_sum = np.zeros((len(labels), len(labels)))
    accuracies = []
    for X_train, X_test, y_train, y_test in k_fold_cross_validation(X, y):
        y_pred = min_distance_classifier(X_train, y_train, X_test)
        cm, acc = evaluate_model(y_test, y_pred, labels)
        conf_matrix_sum += cm
        accuracies.append(acc)
        
    print("\nDistancia Mínima - K-Fold")
    print("Matriz de Confusión (Acumulada):\n", conf_matrix_sum)
    print("Precisión Promedio:", np.mean(accuracies))
    
    # Validación Leave-One-Out
    conf_matrix_sum = np.zeros((len(labels), len(labels)))
    accuracies = []
    for X_train, X_test, y_train, y_test in leave_one_out_validation(X, y):
        y_pred = min_distance_classifier(X_train, y_train, X_test)
        cm, acc = evaluate_model(y_test, y_pred, labels)
        conf_matrix_sum += cm
        accuracies.append(acc)
        
    print("\nDistancia Mínima - Leave-One-Out")
    print("Matriz de Confusión (Acumulada):\n", conf_matrix_sum)
    print("Precisión Promedio:", np.mean(accuracies))



--- Parte I: Clasificador de Distancia Mínima - Iris Dataset ---

Distancia Mínima - Hold-Out
Matriz de Confusión:
 [[15  0  0]
 [ 0 14  1]
 [ 0  3 12]]
Precisión: 0.9111111111111111

Distancia Mínima - K-Fold
Matriz de Confusión (Acumulada):
 [[50.  0.  0.]
 [ 0. 45.  5.]
 [ 0.  7. 43.]]
Precisión Promedio: 0.9200000000000002

Distancia Mínima - Leave-One-Out
Matriz de Confusión (Acumulada):
 [[50.  0.  0.]
 [ 0. 45.  5.]
 [ 0.  7. 43.]]
Precisión Promedio: 0.92

--- Parte I: Clasificador de Distancia Mínima - Wine Dataset ---

Distancia Mínima - Hold-Out
Matriz de Confusión:
 [[15  0  3]
 [ 0 14  7]
 [ 0  5 10]]
Precisión: 0.7222222222222222

Distancia Mínima - K-Fold
Matriz de Confusión (Acumulada):
 [[50.  0.  9.]
 [ 3. 49. 19.]
 [ 1. 17. 30.]]
Precisión Promedio: 0.7245098039215687

Distancia Mínima - Leave-One-Out
Matriz de Confusión (Acumulada):
 [[50.  0.  9.]
 [ 3. 49. 19.]
 [ 1. 17. 30.]]
Precisión Promedio: 0.7247191011235955

--- Parte I: Clasificador de Distancia Mínima -

In [ ]:
# Implementación del clasificador 1NN
def one_nn_classifier(X_train, y_train, X_test):
    knn = KNeighborsClassifier(n_neighbors=1)
    knn.fit(X_train, y_train)
    return knn.predict(X_test)

In [23]:
# Parte II: Clasificador 1NN
for name, (X, y) in datasets.items():
    labels = np.unique(y)  # Definir etiquetas únicas para cada conjunto de datos
    print(f"\n--- Parte II: Clasificador 1NN - {name} Dataset ---")
    
    # Validación Hold-Out
    X_train, X_test, y_train, y_test = holdout_validation(X, y)
    y_pred = one_nn_classifier(X_train, y_train, X_test)
    conf_matrix, accuracy = evaluate_model(y_test, y_pred, labels)
    
    print("\n1NN - Hold-Out")
    print("Matriz de Confusión:\n", conf_matrix)
    print("Precisión:", accuracy)
    
    # Validación K-Fold
    conf_matrix_sum = np.zeros((len(labels), len(labels)))
    accuracies = []
    for X_train, X_test, y_train, y_test in k_fold_cross_validation(X, y):
        y_pred = one_nn_classifier(X_train, y_train, X_test)
        cm, acc = evaluate_model(y_test, y_pred, labels)
        conf_matrix_sum += cm
        accuracies.append(acc)
        
    print("\n1NN - K-Fold")
    print("Matriz de Confusión (Acumulada):\n", conf_matrix_sum)
    print("Precisión Promedio:", np.mean(accuracies))
    
    # Validación Leave-One-Out
    conf_matrix_sum = np.zeros((len(labels), len(labels)))
    accuracies = []
    for X_train, X_test, y_train, y_test in leave_one_out_validation(X, y):
        y_pred = one_nn_classifier(X_train, y_train, X_test)
        cm, acc = evaluate_model(y_test, y_pred, labels)
        conf_matrix_sum += cm
        accuracies.append(acc)
        
    print("\n1NN - Leave-One-Out")
    print("Matriz de Confusión (Acumulada):\n", conf_matrix_sum)
    print("Precisión Promedio:", np.mean(accuracies))



--- Parte II: Clasificador 1NN - Iris Dataset ---

1NN - Hold-Out
Matriz de Confusión:
 [[15  0  0]
 [ 0 15  0]
 [ 0  3 12]]
Precisión: 0.9333333333333333

1NN - K-Fold
Matriz de Confusión (Acumulada):
 [[50.  0.  0.]
 [ 0. 47.  3.]
 [ 0.  3. 47.]]
Precisión Promedio: 0.9600000000000002

1NN - Leave-One-Out
Matriz de Confusión (Acumulada):
 [[50.  0.  0.]
 [ 0. 47.  3.]
 [ 0.  3. 47.]]
Precisión Promedio: 0.96

--- Parte II: Clasificador 1NN - Wine Dataset ---

1NN - Hold-Out
Matriz de Confusión:
 [[14  3  1]
 [ 1 15  5]
 [ 1  5  9]]
Precisión: 0.7037037037037037

1NN - K-Fold
Matriz de Confusión (Acumulada):
 [[50.  5.  4.]
 [ 5. 53. 13.]
 [ 3. 18. 27.]]
Precisión Promedio: 0.7300653594771241

1NN - Leave-One-Out
Matriz de Confusión (Acumulada):
 [[52.  3.  4.]
 [ 5. 54. 12.]
 [ 3. 14. 31.]]
Precisión Promedio: 0.7696629213483146

--- Parte II: Clasificador 1NN - Breast Cancer Dataset ---

1NN - Hold-Out
Matriz de Confusión:
 [[ 58   6]
 [  7 100]]
Precisión: 0.9239766081871345

1NN 